In [1]:
%cd ..
%load_ext autoreload
%autoreload 2
# %load_ext nb_black

from pathlib import Path
import sys

sys.path.insert(0, Path().absolute().parent.as_posix())

/Users/ruizhechao/Documents/NNforHJB


/Users/ruizhechao/Documents/NNforHJB/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Test with Van der Pol equation

In [2]:
# load the data
import numpy as np

path = 'rawdata/raw_data/VDP_beta_0.1_grid_30x30.npy'
data = np.load(path)

# Shuffle once to avoid an ordered train/val split (deterministic)
rng = np.random.default_rng(0)
perm = rng.permutation(len(data))
data = data[perm]

data.dtype
# logger.info(f"Loaded data with shape: {data.shape}, dtype: {data.dtype}")


dtype([('x', '<f8', (2,)), ('dv', '<f8', (2,)), ('v', '<f8')])

#### SSN(trust-region) method for outer weights ##


In [3]:
# Initialize the parameter
power = 2.1
num_neurons = 10 # number greedy insertion selected
num_iterations = 10
loss_weights = (1.0, 1.0)
pruning_threshold = 1e-15

gamma = 5.0
alpha = 1e-5
lr_adam = 1e-5
regularization = (gamma, alpha) 
th = 0.0

In [5]:
from scr.model import model
import torch

test = model(
    alpha, 
    gamma,
    optimizer='SSN',
    activation=torch.relu, 
    train_outerweights=True
    )



2026-01-11 18:01:59.137 | INFO     | scr.model:__init__:71 - Model initialized


In [6]:
# prepare the data
# data is a structured numpy array with fields: 'x', 'dv', 'v'
# convert to the dict format expected by model._prepare_data

data_dict = {
    "x": np.asarray(data["x"], dtype=np.float64),    # shape (N, 2)
    "v": np.asarray(data["v"], dtype=np.float64),    # shape (N,)
    "dv": np.asarray(data["dv"], dtype=np.float64),  # shape (N, 2)
}

data_train, data_valid = test._prepare_data(data_dict)

2026-01-11 18:02:02.176 | INFO     | scr.model:_prepare_data:101 - Training set: 810 samples, Validation set: 90 samples


In [7]:
import torch
from scr.PDPA import PDPA

pdpa = PDPA()

W_hidden, b_hidden = pdpa.sample_uniform_sphere_points(num_neurons)  # numpy arrays

W_hidden_t = torch.as_tensor(W_hidden, dtype=torch.float64)
b_hidden_t = torch.as_tensor(b_hidden, dtype=torch.float64).reshape(-1)  # bias must be (M,)

test.train(
    data_train, 
    data_valid, 
    inner_weights=W_hidden_t, 
    inner_bias=b_hidden_t,
    iterations = 300,
    display_every=100
    )

2026-01-11 18:03:06.328 | INFO     | scr.model:_setup_optimizer:173 - Using SSN optimizer with alpha=1e-05, gamma=5.0, th=0.5, lr =1.0
2026-01-11 18:03:06.329 | INFO     | scr.model:train:252 - Starting network training session
2026-01-11 18:03:06.348 | INFO     | scr.model:train:316 - Epoch 0: Train Loss = 44.800209, Val Loss = 295.504360
2026-01-11 18:03:06.848 | INFO     | scr.model:train:316 - Epoch 100: Train Loss = 8.325550, Val Loss = 16.089095
2026-01-11 18:03:07.382 | INFO     | scr.model:train:316 - Epoch 200: Train Loss = 8.325536, Val Loss = 16.064909
2026-01-11 18:03:07.894 | INFO     | scr.model:train:325 - Best validation loss: 15.054827 at iteration 37


### Test with f(x1, x2) = exp( -(x1**2 + x2 ** 2)/2) * cos(10*x1*x2)

In [27]:
# load the data
import numpy as np

path = 'rawdata/raw_data/gauss_cos_31x31.npy'
data = np.load(path)
data.dtype
# logger.info(f"Loaded data with shape: {data.shape}, dtype: {data.dtype}")

dtype([('x', '<f8', (2,)), ('dv', '<f8', (2,)), ('v', '<f8')])

In [28]:
# Initialize the parameter
power = 2.1
num_neurons = 100 # number greedy insertion selected
loss_weights = (1.0, 1.0)
pruning_threshold = 1e-15

gamma = 5.0
alpha = 1e-5
lr_adam = 1e-5
regularization = (gamma, alpha) 
th = 0.0

In [ ]:
from scr.model import model
import torch

test = model(
    alpha, 
    gamma,
    optimizer='SSN',
    activation=torch.relu, 
    train_outerweights=True
    )


2026-01-11 22:44:48.684 | INFO     | scr.model:__init__:71 - Model initialized


In [30]:
# prepare the data
# data is a structured numpy array with fields: 'x', 'dv', 'v'
# convert to the dict format expected by model._prepare_data

data_dict = {
    "x": np.asarray(data["x"], dtype=np.float64),    # shape (N, 2)
    "v": np.asarray(data["v"], dtype=np.float64),    # shape (N,)
    "dv": np.asarray(data["dv"], dtype=np.float64),  # shape (N, 2)
}

data_train, data_valid = test._prepare_data(data_dict)

2026-01-11 22:44:49.167 | INFO     | scr.model:_prepare_data:101 - Training set: 864 samples, Validation set: 97 samples


In [31]:
import torch

W_hidden, b_hidden = pdpa.sample_uniform_sphere_points(num_neurons)  # numpy arrays

W_hidden_t = torch.as_tensor(W_hidden, dtype=torch.float64)
b_hidden_t = torch.as_tensor(b_hidden, dtype=torch.float64).reshape(-1)  # bias must be (M,)

test.train(
    data_train, 
    data_valid, 
    inner_weights=W_hidden_t, 
    inner_bias=b_hidden_t,
    iterations = 300,
    display_every=100
    )

2026-01-11 22:44:49.673 | INFO     | scr.model:_create_network:140 - Creating network with 100 neurons
2026-01-11 22:44:49.675 | INFO     | scr.model:train:252 - Starting network training session


2026-01-11 22:44:49.695 | INFO     | scr.model:train:316 - Epoch 0: Train Loss = 8.351840, Val Loss = 1553.917378
2026-01-11 22:44:49.855 | INFO     | scr.model:train:316 - Epoch 100: Train Loss = 6.421119, Val Loss = 127.579258
2026-01-11 22:44:50.004 | INFO     | scr.model:train:316 - Epoch 200: Train Loss = 6.168996, Val Loss = 118.968100
2026-01-11 22:44:50.147 | INFO     | scr.model:train:325 - Best validation loss: 83.637147 at iteration 15
